In [ ]:
import tkinter as tk
import math
import random

class RangoliGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Rangoli Puzzle Challenge")

        self.canvas_width = 500
        self.canvas_height = 500
        self.dot_radius = 10

        self.canvas = tk.Canvas(root, width=self.canvas_width, height=self.canvas_height, bg="white")
        self.canvas.pack()

        self.status_label = tk.Label(root, text="", font=("Helvetica", 14))
        self.status_label.pack()

        self.level = 0
        self.total_levels = 10
        self.boards = self.generate_boards(self.total_levels)

        self.dots = []
        self.visited_dots = set()
        self.last_x = None
        self.last_y = None
        self.game_active = False

        # Bind mouse
        self.canvas.bind("<ButtonPress-1>", self.start_game)
        self.canvas.bind("<B1-Motion>", self.track_mouse)
        self.canvas.bind("<ButtonRelease-1>", self.end_game)

        self.load_level()

    def generate_boards(self, count):
        boards = []
        for _ in range(count):
            layout_type = random.choice(["grid", "diamond", "circle", "zigzag"])
            if layout_type == "grid":
                rows = random.randint(3, 5)
                cols = random.randint(3, 5)
                spacing = 80
                board = []
                offset_x = self.canvas_width // 2 - (cols - 1) * spacing // 2
                offset_y = self.canvas_height // 2 - (rows - 1) * spacing // 2
                for i in range(rows):
                    for j in range(cols):
                        x = offset_x + j * spacing
                        y = offset_y + i * spacing
                        board.append((x, y))
                boards.append(board)

            elif layout_type == "diamond":
                size = 4
                board = []
                cx, cy = self.canvas_width // 2, self.canvas_height // 2
                for i in range(-size, size + 1):
                    for j in range(-size, size + 1):
                        if abs(i) + abs(j) <= size:
                            x = cx + j * 40
                            y = cy + i * 40
                            board.append((x, y))
                boards.append(board)

            elif layout_type == "circle":
                board = []
                center_x, center_y = self.canvas_width // 2, self.canvas_height // 2
                r = 150
                points = random.randint(8, 14)
                for i in range(points):
                    angle = 2 * math.pi * i / points
                    x = center_x + r * math.cos(angle)
                    y = center_y + r * math.sin(angle)
                    board.append((x, y))
                boards.append(board)

            elif layout_type == "zigzag":
                board = []
                start_x = 100
                start_y = 150
                for i in range(10):
                    x = start_x + i * 35
                    y = start_y + ((-1) ** i) * 40
                    board.append((x, y))
                boards.append(board)

        return boards

    def load_level(self):
        self.canvas.delete("all")
        self.dots = []
        self.visited_dots = set()
        self.last_x = None
        self.last_y = None

        if self.level >= self.total_levels:
            self.status_label.config(text="🏆 You won all levels! Champion of Rangoli!")
            return

        board = self.boards[self.level]
        for x, y in board:
            dot_id = self.canvas.create_oval(
                x - self.dot_radius, y - self.dot_radius,
                x + self.dot_radius, y + self.dot_radius,
                fill="blue", outline="black"
            )
            self.dots.append((x, y, dot_id))

        self.status_label.config(text=f"Level {self.level + 1} of {self.total_levels}")

    def start_game(self, event):
        self.game_active = True
        self.last_x = event.x
        self.last_y = event.y
        self.visited_dots.clear()
        for _, _, dot_id in self.dots:
            self.canvas.itemconfig(dot_id, fill="blue")

    def track_mouse(self, event):
        if not self.game_active:
            return

        self.canvas.create_line(self.last_x, self.last_y, event.x, event.y, fill="purple", width=2)
        self.last_x, self.last_y = event.x, event.y

        for x, y, dot_id in self.dots:
            if (x, y) not in self.visited_dots:
                if math.hypot(event.x - x, event.y - y) <= self.dot_radius + 5:
                    self.visited_dots.add((x, y))
                    self.canvas.itemconfig(dot_id, fill="green")

    def end_game(self, event):
        self.game_active = False
        if len(self.visited_dots) == len(self.dots):
            self.status_label.config(text=f"✅ Level {self.level + 1} complete!")
            self.level += 1
            self.root.after(1000, self.load_level)
        else:
            self.status_label.config(text=f"❌ Game Over at Level {self.level + 1}")
            self.canvas.create_text(self.canvas_width//2, 20, text="Try Again!", fill="red", font=("Helvetica", 16, "bold"))

# Start the game
root = tk.Tk()
game = RangoliGame(root)
root.mainloop()
